In [2]:
import numpy as np

#16天的窗口,可以分不同窗口生成特征,修改下面的窗口参数即可
label_file = 'V6_labels__6_10.txt' 
train_data_file = 'V6_train_data__6_10.txt'
userId_file = 'V6_userId__6_10.txt'
author_id_file = 'V6_train_authorId__6_10'

reg_log = '/mnt/datasets/fusai/user_register_log.txt'
video_log = '/mnt/datasets/fusai/video_create_log.txt'
launch_log = '/mnt/datasets/fusai/app_launch_log.txt'
act_log = '/mnt/datasets/fusai/user_activity_log.txt'


#设置窗口
TRAIN_PREDICT_DAY = range(10,17)#预测窗口,7天
TRAIN_REGISTER_DAT = range(1,10)#注册时间段
TRAIN_ACT_DAT = range(-6,10)#行为时间段

#测试窗口,包含全部数据,一般不需要修改
TEST_PREDICT_DAY = range(31,38)#预测窗口,7天
TEST_REGISTER_DAT = range(1,31)#注册时间段
TEST_ACT_DAT = range(15,31)#行为时间段
weekend = [6,7,13,14,20,21,27,28]

WEIGHT = np.array([i for i in range(len(TEST_ACT_DAT))])

In [3]:
#求向量中连续非0值的最大长度
def conseq(arr):
    if arr[0] == 0:
        count = 0
        tmp = 0
    else:
        count = 1
        tmp = 1
    for i in range(1,arr.shape[0]):
        if arr[i] == 0:
            tmp = 0
        else:
            tmp += 1
            count = max(count,tmp)
    return count

In [4]:
import pandas as pd

In [5]:
#注册日期
train_register = {}
test_register = {}
#注册设备
train_dev_type = {}
test_dev_type = {}
#注册类型
train_reg_type = {}
test_reg_type = {}

#被转发点赞数
train_act_num = {}
test_act_num = {}

#weekend or not
train_week_workday = {}
test_week_workday = {}

ur = pd.read_csv(reg_log,sep='\t',header=None)
D=dict(ur[3].value_counts())
#ur['dev_type'] = ur[3].apply(lambda x:D[x])
#print('dev type is ready!')
count = 0

In [6]:
#提取用户的注册信息
with open(reg_log,'r') as f:
    while True:
        count += 1
        if count % 50000 == 0:
            print(count)
        line = f.readline()
        if not line:
            break
        tmp = line.strip().split()
        #train_week_workday[tmp[0]] = np.zeros((2,))
        #test_week_workday[tmp[0]] = np.zeros((2,))
        if int(tmp[1]) in TRAIN_REGISTER_DAT:
            train_register[tmp[0]] = int(tmp[1])
            train_dev_type[tmp[0]] = int(tmp[3])
            train_reg_type[tmp[0]] = np.zeros((12,))
            train_reg_type[tmp[0]][int(tmp[2])] = 1
            train_week_workday[tmp[0]] = np.zeros((2,))
            train_act_num[tmp[0]] = 0
        if int(tmp[1]) in TEST_REGISTER_DAT:
            test_register[tmp[0]] = int(tmp[1])
            test_dev_type[tmp[0]] = int(tmp[3])
            test_reg_type[tmp[0]] = np.zeros((12,))
            test_reg_type[tmp[0]][int(tmp[2])] = 1
            test_act_num[tmp[0]] = 0
            test_week_workday[tmp[0]] = np.zeros((2,))

In [7]:
print('train data len is: '+str(len(train_register)))
print('test data len is: '+str(len(test_register)))

In [8]:
#启动日期
train_launch = {}
test_launch = {}
#最早和最晚的启动日期
max_min_train_launch = {}
max_min_test_launch = {}

#标记,是否活跃
labels = {}
for Id in train_register:
    train_launch[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)
    max_min_train_launch[Id] = np.array([0,32])
    labels[Id] = 0

for Id in test_register:
    max_min_test_launch[Id] = np.array([0,32])
    test_launch[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)

In [9]:
#提取用户的启动信息
with open(launch_log,'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        tmp = line.strip().split()
        if tmp[0] in train_register and int(tmp[1]) in TRAIN_PREDICT_DAY:
            labels[tmp[0]] = 1
        if tmp[0] in train_register and int(tmp[1]) in TRAIN_ACT_DAT:
            max_min_train_launch[tmp[0]][0] = max(max_min_train_launch[tmp[0]][0],
                                                  TRAIN_PREDICT_DAY[0]-int(tmp[1]))
            max_min_train_launch[tmp[0]][1] = min(max_min_train_launch[tmp[0]][1],
                                                  TRAIN_PREDICT_DAY[0]-int(tmp[1]))
            train_launch[tmp[0]][int(tmp[1])-TRAIN_ACT_DAT[0]] += 1
            if int(tmp[1]) in weekend:
                train_week_workday[tmp[0]][0] += 1
            else:
                train_week_workday[tmp[0]][1] += 1
                
        if tmp[0] in test_register and int(tmp[1]) in TEST_ACT_DAT:
            max_min_test_launch[tmp[0]][0] = max(max_min_test_launch[tmp[0]][0],
                                                  TEST_PREDICT_DAY[0]-int(tmp[1]))
            max_min_test_launch[tmp[0]][1] = min(max_min_test_launch[tmp[0]][1],
                                                  TEST_PREDICT_DAY[0]-int(tmp[1]))
            test_launch[tmp[0]][int(tmp[1])-TEST_ACT_DAT[0]] += 1
            if int(tmp[1]) in weekend:
                test_week_workday[tmp[0]][0] += 1
            else:
                test_week_workday[tmp[0]][1] += 1

In [10]:
#某天创建了多少个视频
train_video = {}
test_video = {}
#视频创建的最早和最晚日期
max_min_train_video = {}
max_min_test_video = {}
#启动app并且创建了视频的日期
launch_video_train = {}
launch_video_test = {}

for Id in train_register:
    max_min_train_video[Id] = np.array([0,32])
    train_video[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)
    #launch_video_train[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)

for Id in test_register:
    max_min_test_video[Id] = np.array([0,32])
    test_video[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)
    #launch_video_test[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)

In [11]:
#提取用户创建视频的信息
with open(video_log,'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        tmp = line.strip().split()
        if tmp[0] in train_register and int(tmp[1]) in TRAIN_PREDICT_DAY:
            labels[tmp[0]] = 1
        if tmp[0] in train_register and int(tmp[1]) in TRAIN_ACT_DAT:
            max_min_train_video[tmp[0]][0] = max(max_min_train_video[tmp[0]][0],
                                                  TRAIN_PREDICT_DAY[0]-int(tmp[1]))
            max_min_train_video[tmp[0]][1] = min(max_min_train_video[tmp[0]][1],
                                                  TRAIN_PREDICT_DAY[0]-int(tmp[1]))
            train_video[tmp[0]][int(tmp[1])-TRAIN_ACT_DAT[0]] += 1
            if int(tmp[1]) in weekend:
                train_week_workday[tmp[0]][0] += 1
            else:
                train_week_workday[tmp[0]][1] += 1

        if tmp[0] in test_register and int(tmp[1]) in TEST_ACT_DAT:
            max_min_test_video[tmp[0]][0] = max(max_min_test_video[tmp[0]][0],
                                                  TEST_PREDICT_DAY[0]-int(tmp[1]))
            max_min_test_video[tmp[0]][1] = min(max_min_test_video[tmp[0]][1],
                                                  TEST_PREDICT_DAY[0]-int(tmp[1]))
            test_video[tmp[0]][int(tmp[1])-TEST_ACT_DAT[0]] += 1
            if int(tmp[1]) in weekend:
                test_week_workday[tmp[0]][0] += 1
            else:
                test_week_workday[tmp[0]][1] += 1
        
print('video is ready!')

In [12]:
#用户活动的日期
train_act_day = {}#dict.fromkeys(train_register.keys(),
test_act_day = {}#dict.fromkeys(test_register.keys(),
#用户访问的页面类型
train_page = {}#dict.fromkeys(train_register.keys(),
test_page = {}#dict.fromkeys(test_register.keys(),
#用户的活动类型
train_act_type = {}#dict.fromkeys(train_register.keys(),
test_act_type = {}#dict.fromkeys(test_register.keys(),

#暂时没有提取下面的特征
train_video_id = {}
test_video_id = {}                              #np.zeros((6,)))
train_author_id = {}
test_author_id = {}

for Id in train_register:
    train_act_day[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)
    train_page[Id] = np.zeros((5,),dtype=np.int)
    train_act_type[Id] = np.zeros((6,),dtype=np.int)
    launch_video_train[Id] = train_video[Id]*train_launch[Id]
    train_video_id[Id] = []
    #train_author_id[Id] = []

for Id in test_register:
    test_act_day[Id] = np.zeros((len(TRAIN_ACT_DAT),),dtype=np.int)
    test_page[Id] = np.zeros((5,),dtype=np.int)
    test_act_type[Id] = np.zeros((6,),dtype=np.int)
    launch_video_test[Id] = test_video[Id]*test_launch[Id]
    test_video_id[Id] = []
    #test_author_id[Id] = []    

In [13]:
count = 0
with open(act_log,'r') as f:
    while True:
        line = f.readline()
        count += 1
        if count%1000000==0:
            print(count)
        if not line:
            break
        tmp = line.strip().split()
        train_act_num[tmp[4]] = 0
        #train_page_act_num[tmp[4]] = np.zeros((5,))
        #train_type_act_num[tmp[4]] = np.zeros((6,))
        test_act_num[tmp[4]] = 0
        #test_page_act_num[tmp[4]] = np.zeros((5,))
        #test_type_act_num[tmp[4]] = np.zeros((6,))

In [14]:
#提取用户的活动信息
count = 0
with open(act_log,'r') as f:
    while True:
        line = f.readline()
        count += 1
        if count%1000000==0:
            print(count)
        if not line:
            break
        tmp = line.strip().split()
        if tmp[0] in train_register and int(tmp[1]) in TRAIN_PREDICT_DAY:
            labels[tmp[0]] = 1
        if tmp[0] in train_register and int(tmp[1]) in TRAIN_ACT_DAT:
            train_act_day[tmp[0]][int(tmp[1])-TRAIN_ACT_DAT[0]] += 1
            train_page[tmp[0]][int(tmp[2])] += 1
            train_act_type[tmp[0]][int(tmp[-1])] += 1
            train_act_num[tmp[4]] += 1
            if int(tmp[1]) in weekend:
                train_week_workday[tmp[0]][0] += 1
            else:
                train_week_workday[tmp[0]][1] += 1
                
        if tmp[0] in test_register and int(tmp[1]) in TEST_ACT_DAT:
            test_act_day[tmp[0]][int(tmp[1])-TEST_ACT_DAT[0]] += 1
            test_page[tmp[0]][int(tmp[2])] += 1
            test_act_type[tmp[0]][int(tmp[-1])] += 1
            test_act_num[tmp[4]] += 1
            if int(tmp[1]) in weekend:
                test_week_workday[tmp[0]][0] += 1
            else:
                test_week_workday[tmp[0]][1] += 1

print('activity is ready!')

In [15]:
#特征拼接
train_author = []
test_author = []
g = open('/home/kesci/data/'+userId_file,'w')
#af = open('/home/kesci/data/'+author_id_file,'w')
count = 0
y = []
train_data = []
for Id in train_register:
    count = count + 1
    if count % 100000 == 0:
        print(count)
    g.write(Id+'\n')
    y.append(labels[Id])
    #af.write(" ".join(train_author_id[Id])+'\n')
    tmp = np.hstack((np.array([TRAIN_PREDICT_DAY[0]-train_register[Id]]),#0:注册日离预测日的天数
                     train_reg_type[Id],#1~12:注册类型
                     np.array([train_dev_type[Id],D[train_dev_type[Id]]]),#13,14:设备类型
                     #15,16,17,18:app启动的统计信息
                     np.array([sum(train_launch[Id]),sum(train_launch[Id])*1.0/(max_min_train_launch[Id][0]+1),np.std(train_launch[Id]),conseq(train_launch[Id])]),
                     #19,20,21:加权后的app启动信息
                     np.array([sum(train_launch[Id]*WEIGHT),sum(train_launch[Id]*WEIGHT)*1.0/(max_min_train_launch[Id][0]+1),np.std(train_launch[Id]*WEIGHT)]),
                     #22~37:app16天里的启动情况
                     train_launch[Id],
                     #38,39:最早最晚启动的两次
                     max_min_train_launch[Id],
                     #40~45:video创建的统计信息
                     np.array([sum(train_video[Id]),sum(train_video[Id])*1.0/(max_min_train_launch[Id][0]+1),np.std(train_video[Id]),np.max(train_video[Id]),conseq(train_video[Id])]),
                     #46,47:最早最晚创建视频的时间
                     max_min_train_video[Id],
                     #48~52:加权后创建视频的统计信息
                     np.array([sum(train_video[Id]*WEIGHT),sum(train_video[Id]*WEIGHT)*1.0/(max_min_train_launch[Id][0]+1),np.std(train_video[Id]*WEIGHT),np.max(train_video[Id]*WEIGHT)]),
                     #53~68:创建视频的信息
                     train_video[Id],
                     np.array([sum(train_act_day[Id]),sum(train_act_day[Id])*1.0/(max_min_train_launch[Id][0]+1),np.std(train_act_day[Id]),np.max(train_act_day[Id]),np.min(train_act_day[Id]),conseq(train_act_day[Id])]),
                     np.array([sum(train_act_day[Id]*WEIGHT),sum(train_act_day[Id]*WEIGHT)*1.0/(max_min_train_launch[Id][0]+1),np.std(train_act_day[Id]*WEIGHT),np.max(train_act_day[Id]*WEIGHT),np.min(train_act_day[Id]*WEIGHT)]),
                     train_act_day[Id],
                     np.array([sum(train_page[Id]),sum(train_page[Id])*1.0/(max_min_train_launch[Id][0]+1),np.std(train_page[Id]),np.max(train_page[Id]),np.min(train_page[Id])]),
                     train_page[Id],
                     np.array([sum(train_page[Id]!=0)]),
                     np.array([sum(train_act_type[Id]),sum(train_act_type[Id])*1.0/(max_min_train_launch[Id][0]+1),np.std(train_act_type[Id]),np.max(train_act_type[Id]),np.min(train_act_type[Id])]),
                     train_act_type[Id],
                     np.array([sum(train_act_type[Id]!=0)]),
                     np.array([train_act_num[Id]]),
                     train_week_workday[Id]
                     ))
    train_data.append(tmp)
g.close()
#af.close()

In [16]:
np.savetxt('/home/kesci/data/'+label_file,np.array(y))
np.savetxt('/home/kesci/data/'+train_data_file,np.array(train_data))

In [51]:
f = open('/home/kesci/data/V6_userId.txt','w')
#af = open('/home/kesci/data/V5_test_authorId.txt','w')
print('train data is ready!')
test_data = []
count = 0
for Id in test_register:
    count += 1
    if count % 100000 == 0:
        print(count)
    f.write(Id+'\n')
    #af.write(" ".join(test_author_id[Id])+'\n')
    tmp = np.hstack((np.array([TEST_PREDICT_DAY[0]-test_register[Id]]),
                     test_reg_type[Id],
                     np.array([test_dev_type[Id],D[test_dev_type[Id]]]),
                     np.array([sum(test_launch[Id]),sum(test_launch[Id])*1.0/(max_min_test_launch[Id][0]+1),np.std(test_launch[Id]),conseq(test_launch[Id])]),
                     np.array([sum(test_launch[Id]*WEIGHT),sum(test_launch[Id]*WEIGHT)*1.0/(max_min_test_launch[Id][0]+1),np.std(test_launch[Id]*WEIGHT)]),
                     test_launch[Id],
                     max_min_test_launch[Id],
                     np.array([sum(test_video[Id]),sum(test_video[Id])*1.0/(max_min_test_launch[Id][0]+1),np.std(test_video[Id]),np.max(test_video[Id]),conseq(test_video[Id])]),
                     max_min_test_video[Id],
                     np.array([sum(test_video[Id]*WEIGHT),sum(test_video[Id]*WEIGHT)*1.0/(max_min_test_launch[Id][0]+1),np.std(test_video[Id]*WEIGHT),np.max(test_video[Id]*WEIGHT)]),
                     test_video[Id],
                     np.array([sum(test_act_day[Id]),sum(test_act_day[Id])*1.0/(max_min_test_launch[Id][0]+1),np.std(test_act_day[Id]),np.max(test_act_day[Id]),np.min(test_act_day[Id]),conseq(test_act_day[Id])]),
                     np.array([sum(test_act_day[Id]*WEIGHT),sum(test_act_day[Id]*WEIGHT)*1.0/(max_min_test_launch[Id][0]+1),np.std(test_act_day[Id]*WEIGHT),np.max(test_act_day[Id]*WEIGHT),np.min(test_act_day[Id]*WEIGHT)]),
                     test_act_day[Id],
                     np.array([sum(test_page[Id]),sum(test_page[Id])*1.0/(max_min_test_launch[Id][0]+1),np.std(test_page[Id]),np.max(test_page[Id]),np.min(test_page[Id])]),
                     test_page[Id],
                     np.array([sum(test_page[Id]!=0)]),
                     np.array([sum(test_act_type[Id]),sum(test_act_type[Id])*1.0/(max_min_test_launch[Id][0]+1),np.std(test_act_type[Id]),np.max(test_act_type[Id]),np.min(test_act_type[Id])]),
                     test_act_type[Id],
                     np.array([sum(test_act_type[Id]!=0)]),
                     np.array([test_act_num[Id]]),
                     test_week_workday[Id]
                     ))   
    test_data.append(tmp) 
print('all is ready!')
f.close()
#af.close()
np.savetxt('/home/kesci/data/V6_test_data.txt',np.array(test_data))